# 🎬 Movie Discovery Assistant - Fine-tuning with Unsloth

This notebook fine-tunes **Llama 3 (8B)** using **Unsloth**, which makes training 2x faster and uses 70% less memory. Perfect for Google Colab's free Tesla T4 GPU.

**Steps:**
1. Install Unsloth
2. Upload your `train.jsonl`
3. Train the model
4. Save adapters

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Upload Dataset
Please upload your `train.jsonl` file.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="train.jsonl", split="train")

# Define the formatting function
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

### 🎮 Interactive Chat (Improved)
Run this cell to chat with your fine-tuned model! (Repetition fixes applied)

In [ ]:
FastLanguageModel.for_inference(model)

print("🎬 Movie Assistant is READY! (Type 'quit' to exit)")
print("-" * 50)

while True:
    user_query = input("You: ")
    if user_query.lower() in ['quit', 'exit']:
        break
    
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            user_query,
            "", # input
            "", # output
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(
        **inputs, 
        max_new_tokens = 256, 
        use_cache = True,
        repetition_penalty = 1.3,  # Fixes repetition
        no_repeat_ngram_size = 2,  # Prevents 2-word phrase loops
        temperature = 0.9,         # More creativity
        top_k = 50,                # Diversity
        top_p = 0.95
    )
    response = tokenizer.batch_decode(outputs)[0]
    
    # Clean up response to show only the assistant's part
    response_clean = response.split("### Response:")[-1].replace(tokenizer.eos_token, "").strip()
    
    print(f"Assistant: {response_clean}")
    print("-" * 50)

### Save Model (Manual Download)
If the API fails, run this cell to download the model as a ZIP file.

In [ ]:
# Option 2: Manual Download (If API fails)
import shutil
from google.colab import files

# Save locally first
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# Zip the folder
shutil.make_archive('my_movie_model', 'zip', 'lora_model')

# Download it
files.download('my_movie_model.zip')
print("✅ Download started! Unzip this and upload files to https://huggingface.co/new")